<a href="https://colab.research.google.com/github/YuliiaChorna1/Data-Science-11.2-Tensorflow-Graphs-Neural-Networks-Auto-differentiation/blob/main/TensorFlow_practice_extralesson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TensorFlow

- TensorFlow is an open-source library developed by Google specifically for Deep Learning.
- uses a dedicated data type `Tensor`, that can be used directly in the construction and training of neural networks. Tensors in TensorFlow are the fundamental data structure used for representing multi-dimensional arrays.
- TensorFlow has robust support for GPU acceleration, making it efficient for training deep learning models on GPUs.

### Installation

https://www.tensorflow.org/install/pip#linux

In [ ]:
# ! pip install tensorflow[and-cuda]
# ERROR: Cannot install tensorflow[and-cuda]==2.16.1, tensorflow[and-cuda]==2.16.2, tensorflow[and-cuda]==2.17.0, tensorflow[and-cuda]==2.17.1 and tensorflow[and-cuda]==2.18.0 because these package versions have conflicting dependencies.

In [ ]:
import tensorflow as tf
import numpy as np

### Check GPU support

https://www.tensorflow.org/api_docs/python/tf/test/is_gpu_available

In [ ]:
tf.test.is_built_with_cuda()

True

In [ ]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Tensor: Constant vs. Variable

- `tf. constant / tf. Tensor` : Constants are immutable, meaning their values cannot be changed after creation. Once you create a constant tensor, its value remains fixed throughout the execution of the program. Use them for example, if you have hyperparameters or fixed values that need to remain constant throughout the training process.
- `tf. Variable`: Variables are mutable, and their values can be changed during the execution of the program. This makes variables suitable for situations where you need to update the values iteratively, such as in training neural network weights. Use variables when you want to represent trainable parameters in your model, like weights and biases in a neural network.

In [ ]:
x1 = tf.constant([[1.0, 2.0]])#, dtype=np.float16)
x1

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[1., 2.]], dtype=float32)>

In [ ]:
x1.numpy()

array([[1., 2.]], dtype=float32)

In [ ]:
x1[0]

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 2.], dtype=float32)>

In [ ]:
y1 = tf.convert_to_tensor([[2.0], [3.0]])
y1

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [3.]], dtype=float32)>

In [ ]:
b1 = tf.constant(4.0)
b1

<tf.Tensor: shape=(), dtype=float32, numpy=4.0>

In [ ]:
b1 = tf.Variable(4.0)
b1

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=4.0>

In [ ]:
import tensorflow as tf

# Using tf.constant
constant_tensor = tf.constant([1, 2, 3])

# Using tf.Variable
variable_tensor = tf.Variable([1, 2, 3])

# You can update the value of a tf.Variable
variable_tensor.assign([4, 5, 6])

# Accessing the values
print(constant_tensor.numpy()) # Output: [1 2 3]
print(variable_tensor.numpy()) # Output: [4 5 6]

[1 2 3]
[4 5 6]


### **Eager vs. Graph execution / Інтерактивне (послідовне) викомання проти Графічного виконання**:

https://www.tensorflow.org/guide/intro_to_graphs

In [ ]:
# https://ww.tensorflow.org/api_docs/python/tf/executing_eagerly
# Default
assert tf.multiply(6, 7).numpy() == 42
tf.executing_eagerly()

True

In [ ]:
# Simple Python function
def f():
    print("Eager execution in a simple Python function:", tf.executing_eagerly())
    print(tf.multiply(6, 7).numpy())

f()

Eager execution in a simple Python function: True
42


In [ ]:
tf.config.run_functions_eagerly(False) # Default behaviour
@tf.function # Compiles a function into a callable TensorFlow graph (https://www.tensorflow.org/api_docs/python/tf/function)
def fn_graph():
    with tf.init_scope(): # context manager, moves clause out of the graph
        print("Eager execution outside of the graph:", tf.executing_eagerly()) # outside the graph
    print("Eager execution inside of the graph:", tf.executing_eagerly()) # inside the graph
    return tf.multiply(6, 7)#.numpy()

fn_graph().numpy()

Eager execution outside of the graph: True
Eager execution inside of the graph: False


42

In [ ]:
fn_graph()

<tf.Tensor: shape=(), dtype=int32, numpy=42>

In [ ]:
tf.config.run_functions_eagerly(True)
@tf.function # Compiles a function into a callable TensorFlow graph (https://www.tensorflow.org/api_docs/python/tf/function)
def fn_eager():
    with tf.init_scope():
        print("Eager execution outside of the graph:", tf.executing_eagerly()) # outside the graph
    print("Eager execution inside of the graph:", tf.executing_eagerly()) # inside the graph
    return tf.multiply(6, 7).numpy()
fn_eager()

Eager execution outside of the graph: True
Eager execution inside of the graph: True


42

In [ ]:
# fn_eager().graph() # недоступний в даному випадку бо fn_eager побудована як послідовна функція

In [ ]:
# Switch to graph execution by default
tf.compat.v1.disable_eager_execution()

In [1]:
# fn_eager() # -> no eager execution anymore because of code above

In [ ]:
def fn():
    print("Eager execution in a simple Python function:", tf.executing_eagerly())

fn()

# Eager execution cannot be enabled after Tensorflow APIs have been used to
# create or execute graphs. It is typically recommended to invoke this function
# at program startup and not in a library (as most (torarzes should de usable
# both with and without eager execution)

Eager execution in a simple Python function: False


*When to choose what?*

**Eager** execution:
- if you are experimenting and debugging is important
- if you have a few computationally expensive operations

**Graph** execution:
- If you need to optimize your code when you have many operations which are in itself not necessarily heavy

*Good practice:*
- use `@tf.function` decorator to define your functions and switch on the option `tf.config.run_functions_eagerly(True)` when debugging to be able to turn it to `False` otherwise if needed.